# Light GBM Model
#### cv score :  0.5502911153119092 (5 folds, tgt enc)
#### cv score :  0.5534669187145557 (5fold)
#### cv score :  0.5534489970795226 (5 even fold)
#### cv score :  0.5507674858223063   (10 folds)
#### cv score :  0.5539240004577335   (5 even folds, nomizumahi)

In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import sklearn
from Functions import prepro
import warnings

import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')
import lightgbm as lgb
from lightgbm import LGBMClassifier


FEATURE_SELECTION = False

random.seed(0)

pd.set_option("display.max_columns", 600)
pd.set_option("display.max_rows", 600)


In [2]:
train_df = pd.read_csv("../../data/Processed/train2.csv")
test_df = pd.read_csv("../../data/Processed/test2.csv")
print(train_df.shape)
print(test_df.shape)

(66125, 192)
(28340, 191)


In [3]:
y = train_df["y"].values
train_df = train_df.drop("y", axis=1)

In [4]:
# add disconnection info

train_df = prepro.add_disconnection(train_df)
test_df = prepro.add_disconnection(test_df)

In [5]:
# add info about numeric column
# 武器データの中のnum_colsを持つ特徴について、チーム内の統計量を計算、levelは結構効いてる

num_cols = ["level", "range", "rapid-", "atack", "ink-sub", "fav-main", "good-special"]

#train_df = prepro.flat(train_df, num_cols)
#test_df = prepro.flat(test_df, num_cols)

train_df = prepro.add_numeric_info(train_df, num_cols)
test_df = prepro.add_numeric_info(test_df, num_cols)

# 欠損値埋め先にやろう
欠損値がある列

rank, weapon, level ⇨　回線切れ or ナワバリ

weaponの欠損に並んでそれに関係ある列、A3, A4, B3, B4 ⇨ 回線切れ

level, weaponが消えていたら回線切れ

In [6]:
# rankの欠損値を埋める
train_df, test_df = prepro.fillna_rank(train_df, test_df)

#そのほかの欠損値を埋める
train_df, test_df = prepro.fillna(train_df, test_df)

In [7]:
print(train_df.isnull().sum().sum())
print(test_df.isnull().sum().sum())

0
0


# 特徴量エンジニアリング


In [8]:
# count reskin, by mode
print("reskin")
train_df, test_df = prepro.count_reskin(train_df, test_df)
train_df, test_df = prepro.count_reskin_by_mode(train_df, test_df)

# count mainweapon, by mode
print("mainweapon")
train_df, test_df = prepro.count_mainweapon(train_df, test_df)
train_df, test_df = prepro.count_mainweapon_by_mode(train_df, test_df)

# count subweapon, by mode
print("subweapon")
train_df, test_df = prepro.count_subweapon(train_df, test_df)
train_df, test_df = prepro.count_subweapon_by_mode(train_df, test_df)

# count special, by mode
print("special")
train_df, test_df = prepro.count_special(train_df, test_df)
train_df, test_df = prepro.count_special_by_mode(train_df, test_df)

#identify A1
train_df, test_df = prepro.identify_A1(train_df, test_df)


# 水増し, A1も統計量に含めた特徴を作る場合は水ましより先にやる
train_df, y = prepro.mizumashi(train_df, y)

# is_nawabari
train_df, test_df = prepro.is_nawabari(train_df, test_df)

# match rank
train_df, test_df = prepro.match_rank(train_df, test_df)



reskin
mainweapon
subweapon
special


In [9]:
# add team info

train_df,  test_df = prepro.addTeamInfo(train_df, test_df, cols=["special", "subweapon", "category1", "category2", "mainweapon"])

(160590, 344)
special-A
special-B
subweapon-A
subweapon-B
category1-A
category1-B
category2-A
category2-B
mainweapon-A
mainweapon-B
(160590, 538)
complete


In [10]:
# categorize team

categorize_col = ["category1", "category2", "subweapon", "special", "mainweapon"]
for col in categorize_col:
    print(col)
    train_df_, test_df_ = prepro.categorize_team(train_df, test_df, col)

category1
category2
subweapon
special
mainweapon


In [11]:
# product categorical feature
train_df, test_df = prepro.prod(train_df, test_df, "mode", "stage")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category1-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category1-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category2-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category2-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-mainweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-mainweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-subweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-subweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-special-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-special-B")

train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category1-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category1-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category2-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category2-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-mainweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-mainweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-subweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-subweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-special-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-special-B")

In [12]:
# make input

drop_cols = ["id", "lobby", "lobby-mode",  "period", "game-ver", "A1-weapon", "A2-weapon", "A3-weapon", "A4-weapon", \
              "B1-weapon", "B2-weapon", "B3-weapon", "B4-weapon"]

categorical_feature = [col for col in train_df.dtypes[train_df.dtypes == "object"].index.to_list() if col not in drop_cols]

X, test_X = prepro.make_input(train_df, test_df, drop_cols, categorical_encode=True, scaler=False, verbose=False)

complete


# 特徴選択

In [13]:
if FEATURE_SELECTION:
    from sklearn.ensemble import RandomForestClassifier

    clf = RandomForestClassifier()
    clf.fit(X, y)
    importance = pd.DataFrame(clf.feature_importances_, index=X.columns, columns=['importance']).sort_values('importance', ascending=False)
    display(importance*10000)
    use_cols = importance.index[:450]
else:
    print("No feature selection")


No feature selection


# モデル学習

In [83]:
cvs = []
train_pred_features = []
test_pred_features = []

for _ in range(10):
    print("iteration {}".format(_))
    
    # 全データを5つに分割
    random.seed(random.randint(0, 10000))
    SIZE = X.shape[0]
    K = 5
    cat_tgtenc_cols = ["mode", "stage", "team-category1-A", "team-category1-B"]
    #cat_tgtenc_cols = ["mode", "stage"]

    # folds = prepro.make_kfolds(SIZE, K)
    #folds = prepro.make_stratified_kfolds(X, y, K)
    folds = prepro.make_even_kfolds(X, y, K)




    if SIZE != len(set(sum(folds, []))):
        print("error is occuring in spliting")
    else :
        print("successfully split")

    if FEATURE_SELECTION:
        X = X[use_cols]
        test_X = test_X[use_cols]
        
    param = {
        "num_leaves" : 28,
        "learning_rate" : 0.01,
        #"learning_rate" : 0.05,
        "num_iterations" : 2000,
        "objective" : "binary",
        "metric" : ["binary_logloss"],
        "random_state" : random.randint(0, 10000),
        "max_depth" : 100
    }

    THRESHOLD = 0.50
    models = []
    cv_scores = []
    temp = 0
    train_pred = []
    train_Xs = []
    tgt_encode = False

    all_indices = sum(folds, [])
    for i in range(K):
        print("======================== fold {} ========================".format(i+1))
        valid_indices = folds[i]
        train_indices = list(set(all_indices) - set(valid_indices))
        # print("train ", len(train_indices), " , valid ", len(valid_indices))
        train_X = X.iloc[train_indices]
        try:
            train_y = y.iloc[train_indices]
        except:
            train_y = y[train_indices]
        valid_X = X.iloc[valid_indices]
        try:
            valid_y = y.iloc[valid_indices]
        except:
            valid_y = y[valid_indices]


        if tgt_encode:
            for col in cat_tgtenc_cols:
                print(col)
                train_X, valid_X = prepro.target_encoding(train_X, valid_X, train_y, col, "y")


        train_data = lgb.Dataset(train_X, label=train_y)
        valid_data = lgb.Dataset(valid_X, label=valid_y)

        model = lgb.train(
            param, 
            train_data, 
            valid_sets=valid_data, 
            #categorical_feature=categorical_feature,
            early_stopping_rounds=40, 
            verbose_eval=1000,

        )
        pred = model.predict(valid_X)
        pred = np.where(pred < THRESHOLD, 0, 1)
        train_pred.append(pred)
        temp += np.sum(pred)

        score = accuracy_score(pred, valid_y)

        models.append(model)
        cv_scores.append(score)


    print("cv score : ", np.mean(cv_scores))    
    print("cv ratio : ", temp / SIZE)
    cvs.append(np.mean(cv_scores))
    
    
    preds = []
    if tgt_encode:
        for col in cat_tgtenc_cols:
            print(col)
            X, test_X = prepro.target_encoding(X, test_X, y, col, "y")

    for i in range(K):
        model = models[i]
        pred = model.predict(test_X)
        preds.append(pred)
        print(np.sum(pred) / pred.shape[0])

    preds = np.array(preds)
    preds = np.mean(preds, axis=0)
    test_pred_features.append(preds) #
    


    train_df["pred"] = 0
    train_df["y"] = y
    for i in range(K):
        train_df["pred"].iloc[folds[i]] = train_pred[i]
    
    train_pred_features.append(train_df["pred"].values) #
    

iteration 0
successfully split
======================== fold 1 ========================
Training until validation scores don't improve for 40 rounds
[1000]	valid_0's binary_logloss: 0.68164
Early stopping, best iteration is:
[1014]	valid_0's binary_logloss: 0.681628
======================== fold 2 ========================
Training until validation scores don't improve for 40 rounds
Early stopping, best iteration is:
[905]	valid_0's binary_logloss: 0.68243
======================== fold 3 ========================
Training until validation scores don't improve for 40 rounds
Early stopping, best iteration is:
[688]	valid_0's binary_logloss: 0.681352
======================== fold 4 ========================
Training until validation scores don't improve for 40 rounds
[1000]	valid_0's binary_logloss: 0.682036
Early stopping, best iteration is:
[1281]	valid_0's binary_logloss: 0.681891
======================== fold 5 ========================
Training until validation scores don't improve for 4

KeyboardInterrupt: 

In [ ]:
for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    index = np.argsort(cvs)[::-1][:i]
    train_pred_f = np.array(train_pred_features)[index]
    test_pred_f = np.array(test_pred_features)[index]
    train_pred_f = np.mean(train_pred_f, axis=0)
    train_pred_f = np.where(train_pred_f < THRESHOLD, 0, 1)
    acc = accuracy_score(pred, valid_y)
    print("{} preds : cv is {}".format(i, acc))

In [72]:
submit_df = pd.DataFrame({'y': pred})
submit_df.index.name = 'id'
submit_df.to_csv('../Submissions/kibounoichigeki_2.csv'.format(K))